<a href="https://colab.research.google.com/github/ciepielajan/SDA_SMOG_Wawelski/blob/main/Jan/SDA_2021_04_18_Projekt_koncowy_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SCHEMAT POD KNN

1. Przygotowanie danych pod model (drzewo)
2. Obliczenie predykcji dla WSZYSTKICH sensorów dla 1.01.2018
3. Trenuje model KNN 
* X - LAT, LONG 
* Y - PM10 z punktu (2) 
4. Pobieram nowe robocze punkty (siatka punktów) 
* Pętla z janka colaba
* Wystarczą same lat, long
5. Predykcja model KNN na lokalizacjach z punktu 4
6. Wrzucam wynik do df  (Kolumny lat, long, predit_pm10)
7. Wrzucam całość na mapę
8. gotowe


In [4]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import pickle

In [1]:
data = "2017-12-03"
time = "11"
user_latitude = "50.057747"
user_longitude = "19.961374"

pobieram sensory

In [22]:
!gdown --id "186S03hFJ67K4eV1BWb2_6lZxAYG7dBM4"
!unzip -q "archive.zip" -d data/

sensors = pd.read_csv("data/sensor_locations.csv")
sensors[:3]

Downloading...
From: https://drive.google.com/uc?id=186S03hFJ67K4eV1BWb2_6lZxAYG7dBM4
To: /content/archive.zip
2.39MB [00:00, 141MB/s]
replace data/april-2017.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


,id,latitude,longitude
0,3,50.057748,19.961375
1,140,50.057747,19.961374
2,142,50.084816,19.878066


pobieram wytrenowany model

In [3]:
!wget -q "https://github.com/ciepielajan/SDA_SMOG_Wawelski/raw/main/Jan/model/pipeline.pickle"

wczytanie modelu

In [6]:
with open('pipeline.pickle', 'rb') as handle:
    pipeline = pickle.load(handle)

pipeline

Pipeline(memory=None,
         steps=[('prep',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('ohe',
                                                  OneHotEncoder(categories='auto',
                                                                drop='first',
                                                                dtype=<class 'numpy.float64'>,
                                                                handle_unknown='error',
                                                                sparse=True),
                                                  ['season',
                                                   'part_of_the_day'])],
                                   verbose=False)),
                ('model',
                 RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
 

pobieram siatkę punktów

In [8]:
points = pd.read_csv("https://raw.githubusercontent.com/ciepielajan/SDA_SMOG_Wawelski/main/Jan/model/points.csv")
points[:3]

,Unnamed: 0,id,latitude,longitude
0,0,1-1,50.10700,19.82
1,1,1-2,50.10403,19.82
2,2,1-3,50.10106,19.82


zaimportowanie funkcji do preprocessingu

In [9]:
def parse_season(x):
    if x in [4,5,6,7,8,9]:
       return "lato"
    elif x in [1,2,3,10,11,12]:
       return "zima"
    else:
       return "bład"

def parse_hours(x):
    if x in [10,11,12,13,14,15,16]:
       return "dzien"
    elif x in [17,18,19,20,21,22,23,0,1,2,3,4,5,6,7,8,9,]:
       return "noc"
    else:
       return "bład"

In [14]:
user_time = pd.Timestamp(data + " " + time + ":00:00")

In [15]:
user_date = pd.DataFrame([user_latitude, user_longitude]).T
user_date.columns = ["latitude","longitude"]
user_date["season"] = parse_season(user_time.month)
user_date["part_of_the_day"] = parse_hours(user_time.hour)
user_date

,latitude,longitude,season,part_of_the_day
0,50.057747,19.961374,zima,dzien


In [24]:
sensors["season"] = parse_season(user_time.month)
sensors["part_of_the_day"] = parse_hours(user_time.hour)
sensors = sensors[["latitude","longitude","season","part_of_the_day"]]
sensors[:3]

,latitude,longitude,season,part_of_the_day
0,50.057748,19.961375,zima,dzien
1,50.057747,19.961374,zima,dzien
2,50.084816,19.878066,zima,dzien


In [35]:
points_prediction = points[["latitude","longitude"]]
# points_prediction["season"] = user_date["season"].values[0]
# points_prediction["part_of_the_day"] = user_date["part_of_the_day"].values[0]
points_prediction[:3]

,latitude,longitude
0,50.10700,19.82
1,50.10403,19.82
2,50.10106,19.82


predykcja na sensorach

In [26]:
#predykcja
predict = pipeline.predict(sensors)
sensors["prediction_pm10"] = predict
sensors[:3]

,latitude,longitude,season,part_of_the_day,prediction_pm10
0,50.057748,19.961375,zima,dzien,49.792832
1,50.057747,19.961374,zima,dzien,49.792832
2,50.084816,19.878066,zima,dzien,72.841786


Trenuje model KNN

X - LAT, LONG  \
Y - PM10 z punktu (2)

In [33]:
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline

In [ ]:
# # define the data preparation for the columns
# t = [
#      ('ohe', OneHotEncoder(drop="first"), ["season","part_of_the_day"]),  #na kategorycznych zastosuj OneHotEncoder
#      ]
# col_transform = ColumnTransformer(transformers=t, remainder="passthrough")   #wykonaj transformacje . parametr remainder definiuje co zrobić z pozostałmi kolumnami
# col_transform


In [ ]:
# pipeline_LinearRegression = Pipeline(steps=[('prep',col_transform), ('model', model_LinearRegression)])   # połącz  transformację kolumn z trenowaniem modelu 
# pipeline_LinearRegression.fit(X,y)

In [29]:
sensors_neigh_X = sensors[["latitude","longitude"]]
sensors_neigh_y = sensors["prediction_pm10"]

In [30]:
from sklearn.neighbors import KNeighborsRegressor
neigh = KNeighborsRegressor()
neigh.fit(sensors_neigh_X, sensors_neigh_y)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                    weights='uniform')

5 Trenuje model KNN na lokalizacjach z punktu 4

In [38]:
points_prediction[:3]

,latitude,longitude
0,50.10700,19.82
1,50.10403,19.82
2,50.10106,19.82


In [39]:
#predykcja
neigh_predict = neigh.predict(points_prediction)
points_prediction["prediction_pm10"] = neigh_predict
points_prediction[:3]

,latitude,longitude,prediction_pm10
0,50.10700,19.82,57.567263
1,50.10403,19.82,57.567263
2,50.10106,19.82,57.567263


In [44]:
points_prediction["prediction_pm10"].describe().to_frame().T

,count,mean,std,min,25%,50%,75%,max
prediction_pm10,1575.0,54.147558,4.419812,43.758499,51.012071,53.720737,56.014393,71.454769


In [45]:
!pip -q install folium
import folium

In [48]:
location = [50.0614 ,19.9366]

m = folium.Map(location=location, zoom_start=13)


for ind in points_prediction.index: 
    tooltip_txt = 'PM10:' + str(points_prediction['prediction_pm10'][ind])
    
    if points_prediction['prediction_pm10'][ind] <= 25:
      color="green",
    elif (points_prediction['prediction_pm10'][ind] > 25) & (points_prediction['prediction_pm10'][ind] <= 50):
      color="orange",
    elif (points_prediction['prediction_pm10'][ind] > 50) & (points_prediction['prediction_pm10'][ind] <= 100):
      color="red",
    elif (points_prediction['prediction_pm10'][ind] > 100):
      color="black",
    else:
      color="white"

    folium.Circle(
          location=[float(points_prediction['latitude'][ind]), float(points_prediction['longitude'][ind])], 
          radius=165,
          # popup="Laurelhurst Park",
          # fill=True,
          # fill_color="#3186cc",
          fill_color=color,
          color=color,
          tooltip= tooltip_txt
      ).add_to(m)
m

In [47]:
name_file = "index.html"

m.save(name_file)
from google.colab import files
files.download(name_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>